# 🧬 Módulo 2: Representación y Visualización Molecular
## Actividad 2.6: Descriptores Moleculares y Propiedades Calculadas

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_02_representacion_molecular/06_descriptores.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender qué son los descriptores moleculares y su importancia
- Calcular descriptores 2D (topológicos, estructurales, electrónicos)
- Calcular descriptores 3D (geométricos, superficiales, volumétricos)
- Generar y comparar fingerprints moleculares
- Estimar propiedades fisicoquímicas (logP, solubilidad, permeabilidad)
- Aplicar descriptores en análisis QSAR/QSPR
- Realizar análisis de similitud molecular
- Visualizar y correlacionar descriptores con actividad biológica

---

## 1. Instalación de Dependencias

In [ ]:
# Instalación en Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit pandas matplotlib seaborn scikit-learn -q

print("✓ Dependencias instaladas")

In [ ]:
# Importar librerías necesarias
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, Lipinski, QED
from rdkit.Chem import rdMolDescriptors, rdPartialCharges
from rdkit.Chem import Draw, DataStructs
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías importadas correctamente")

## 2. Conceptos Fundamentales

### ¿Qué son los Descriptores Moleculares?

Los **descriptores moleculares** son valores numéricos que codifican información estructural, electrónica y topológica de una molécula. Son fundamentales en:

- **QSAR/QSPR**: Relaciones cuantitativas estructura-actividad/propiedad
- **Virtual Screening**: Búsqueda de candidatos a fármacos
- **Predicción de propiedades**: Solubilidad, toxicidad, permeabilidad
- **Similitud molecular**: Comparación y agrupamiento de compuestos

### Clasificación de Descriptores

#### 1. **Descriptores 0D**: Composición elemental
- Fórmula molecular
- Peso molecular
- Conteo de átomos por tipo

#### 2. **Descriptores 1D**: Fragmentos y grupos funcionales
- Grupos funcionales
- Fragmentos estructurales

#### 3. **Descriptores 2D**: Topología molecular
- Índices topológicos (Wiener, Zagreb, Balaban)
- Conectividad molecular
- Conteo de enlaces rotables
- Fingerprints moleculares

#### 4. **Descriptores 3D**: Geometría tridimensional
- Volumen molecular (Van der Waals)
- Superficie accesible al solvente (SASA)
- Momentos dipolares
- Descriptores de forma (PMI, asphericity)

## 3. Descriptores 2D (No dependen de conformación 3D)

### 3.1 Descriptores Básicos y Propiedades Fisicoquímicas

In [ ]:
def calcular_descriptores_basicos(smiles, nombre="Molécula"):
    """
    Calcula descriptores moleculares básicos y propiedades fisicoquímicas
    """
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        print(f"Error: No se pudo procesar el SMILES {smiles}")
        return None
    
    descriptores = {
        'Nombre': nombre,
        'SMILES': smiles,
        
        # Descriptores 0D - Composición
        'Fórmula': rdMolDescriptors.CalcMolFormula(mol),
        'Peso Molecular': Descriptors.MolWt(mol),
        'Peso Molecular Exacto': Descriptors.ExactMolWt(mol),
        
        # Descriptores 1D/2D - Estructura
        'Num Átomos Pesados': Descriptors.HeavyAtomCount(mol),
        'Num Átomos Total': mol.GetNumAtoms(),
        'Num Enlaces': mol.GetNumBonds(),
        'Num Aromáticos': Descriptors.NumAromaticRings(mol),
        'Num Anillos': Descriptors.RingCount(mol),
        'Num Enlaces Rotables': Descriptors.NumRotatableBonds(mol),
        
        # Propiedades Lipinski (Regla de los 5)
        'HBD (Donors)': Descriptors.NumHDonors(mol),
        'HBA (Acceptors)': Descriptors.NumHAcceptors(mol),
        'LogP': Descriptors.MolLogP(mol),
        'TPSA': Descriptors.TPSA(mol),  # Área superficial polar topológica
        
        # Complejidad molecular
        'Complejidad': Descriptors.BertzCT(mol),
        'Sp3 Fraction': Descriptors.FractionCsp3(mol),
        
        # Carga y electrónica (estimadas)
        'Carga Formal': Chem.GetFormalCharge(mol),
    }
    
    return descriptores

# Ejemplo con varios fármacos conocidos
farmacos = [
    ('Aspirina', 'CC(=O)Oc1ccccc1C(=O)O'),
    ('Ibuprofeno', 'CC(C)Cc1ccc(cc1)C(C)C(=O)O'),
    ('Paracetamol', 'CC(=O)Nc1ccc(O)cc1'),
    ('Cafeína', 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'),
]

# Calcular descriptores para cada fármaco
resultados = []
for nombre, smiles in farmacos:
    desc = calcular_descriptores_basicos(smiles, nombre)
    if desc:
        resultados.append(desc)

# Crear DataFrame
df_descriptores = pd.DataFrame(resultados)
df_descriptores

In [ ]:
# Visualizar moléculas
mols = [Chem.MolFromSmiles(smiles) for _, smiles in farmacos]
legends = [nombre for nombre, _ in farmacos]
img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(250, 250), 
                           legends=legends, returnPNG=False)
display(img)

### 3.2 Regla de los 5 de Lipinski (Drug-likeness)

La **Regla de Lipinski** establece criterios para determinar si una molécula tiene propiedades similares a un fármaco oral:

1. **Peso molecular ≤ 500 Da**
2. **LogP ≤ 5** (lipofilia moderada)
3. **Donadores H ≤ 5** (HBD)
4. **Aceptores H ≤ 10** (HBA)
5. **Violaciones ≤ 1** (se permite una violación)

In [ ]:
def evaluar_lipinski(mol):
    """
    Evalúa la Regla de los 5 de Lipinski
    """
    MW = Descriptors.MolWt(mol)
    LogP = Descriptors.MolLogP(mol)
    HBD = Descriptors.NumHDonors(mol)
    HBA = Descriptors.NumHAcceptors(mol)
    
    violaciones = 0
    detalles = []
    
    if MW > 500:
        violaciones += 1
        detalles.append(f"MW = {MW:.1f} > 500")
    
    if LogP > 5:
        violaciones += 1
        detalles.append(f"LogP = {LogP:.1f} > 5")
    
    if HBD > 5:
        violaciones += 1
        detalles.append(f"HBD = {HBD} > 5")
    
    if HBA > 10:
        violaciones += 1
        detalles.append(f"HBA = {HBA} > 10")
    
    cumple = violaciones <= 1
    
    return {
        'MW': MW,
        'LogP': LogP,
        'HBD': HBD,
        'HBA': HBA,
        'Violaciones': violaciones,
        'Cumple': cumple,
        'Detalles': ' | '.join(detalles) if detalles else 'Cumple todas'
    }

# Evaluar fármacos
print("Evaluación de la Regla de Lipinski:\n")
print(f"{'Fármaco':<15} {'MW':<8} {'LogP':<8} {'HBD':<6} {'HBA':<6} {'Viol.':<7} {'Cumple':<10}")
print("="*70)

for nombre, smiles in farmacos:
    mol = Chem.MolFromSmiles(smiles)
    resultado = evaluar_lipinski(mol)
    cumple_str = "✓ Sí" if resultado['Cumple'] else "✗ No"
    print(f"{nombre:<15} {resultado['MW']:<8.1f} {resultado['LogP']:<8.2f} "
          f"{resultado['HBD']:<6} {resultado['HBA']:<6} {resultado['Violaciones']:<7} {cumple_str:<10}")
    if resultado['Detalles'] != 'Cumple todas':
        print(f"  → {resultado['Detalles']}")

### 3.3 Descriptores Topológicos Avanzados

In [ ]:
def calcular_descriptores_topologicos(mol):
    """
    Calcula descriptores topológicos avanzados
    """
    descriptores = {
        # Índices de conectividad
        'Chi0v': Descriptors.Chi0v(mol),
        'Chi1v': Descriptors.Chi1v(mol),
        'Chi2v': Descriptors.Chi2v(mol),
        'Chi3v': Descriptors.Chi3v(mol),
        'Chi4v': Descriptors.Chi4v(mol),
        
        # Índices de Kappa (forma molecular)
        'Kappa1': Descriptors.Kappa1(mol),
        'Kappa2': Descriptors.Kappa2(mol),
        'Kappa3': Descriptors.Kappa3(mol),
        
        # Índice de Balaban (conectividad promedio)
        'BalabanJ': Descriptors.BalabanJ(mol),
        
        # Índice de Wiener (suma de distancias)
        'Wiener': Chem.GraphDescriptors.Ipc(mol),
        
        # Complejidad molecular
        'BertzCT': Descriptors.BertzCT(mol),
        
        # Índice de Lipinski
        'Lipinski': Descriptors.NumHDonors(mol) + Descriptors.NumHAcceptors(mol),
    }
    
    return descriptores

# Calcular para los fármacos
print("Descriptores Topológicos:\n")
topo_data = []

for nombre, smiles in farmacos:
    mol = Chem.MolFromSmiles(smiles)
    desc_topo = calcular_descriptores_topologicos(mol)
    desc_topo['Nombre'] = nombre
    topo_data.append(desc_topo)

df_topo = pd.DataFrame(topo_data)
cols = ['Nombre'] + [col for col in df_topo.columns if col != 'Nombre']
df_topo = df_topo[cols]
df_topo

### 3.4 Fingerprints Moleculares

Los **fingerprints** son representaciones binarias de la estructura molecular, útiles para comparar similitud.

In [ ]:
def generar_fingerprints(mol, tipo='Morgan', radio=2, nbits=2048):
    """
    Genera diferentes tipos de fingerprints moleculares
    
    Tipos disponibles:
    - 'Morgan': Circular fingerprint (ECFP)
    - 'MACCS': MACCS keys (166 bits)
    - 'RDKit': RDKit fingerprint
    - 'AtomPair': Atom-pair fingerprint
    - 'TopologicalTorsion': Torsion fingerprint
    """
    if tipo == 'Morgan':
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radio, nBits=nbits)
    elif tipo == 'MACCS':
        fp = AllChem.GetMACCSKeysFingerprint(mol)
    elif tipo == 'RDKit':
        fp = Chem.RDKFingerprint(mol, maxPath=7, fpSize=nbits)
    elif tipo == 'AtomPair':
        fp = AllChem.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=nbits)
    elif tipo == 'TopologicalTorsion':
        fp = AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect(mol, nBits=nbits)
    else:
        raise ValueError(f"Tipo de fingerprint '{tipo}' no reconocido")
    
    return fp

def calcular_similitud(mol1, mol2, tipo='Morgan', metrica='Tanimoto'):
    """
    Calcula la similitud entre dos moléculas usando fingerprints
    
    Métricas disponibles:
    - 'Tanimoto': Coeficiente de Tanimoto (Jaccard)
    - 'Dice': Coeficiente de Dice
    - 'Cosine': Similitud coseno
    """
    fp1 = generar_fingerprints(mol1, tipo=tipo)
    fp2 = generar_fingerprints(mol2, tipo=tipo)
    
    if metrica == 'Tanimoto':
        sim = DataStructs.TanimotoSimilarity(fp1, fp2)
    elif metrica == 'Dice':
        sim = DataStructs.DiceSimilarity(fp1, fp2)
    elif metrica == 'Cosine':
        sim = DataStructs.CosineSimilarity(fp1, fp2)
    else:
        raise ValueError(f"Métrica '{metrica}' no reconocida")
    
    return sim

# Calcular matriz de similitud
print("Matriz de Similitud (Tanimoto con Morgan Fingerprints):\n")

nombres = [nombre for nombre, _ in farmacos]
mols = [Chem.MolFromSmiles(smiles) for _, smiles in farmacos]
n = len(mols)

# Crear matriz de similitud
matriz_sim = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        matriz_sim[i, j] = calcular_similitud(mols[i], mols[j], tipo='Morgan')

# Mostrar como DataFrame
df_sim = pd.DataFrame(matriz_sim, index=nombres, columns=nombres)
df_sim

In [ ]:
# Visualizar matriz de similitud como heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df_sim, annot=True, fmt='.3f', cmap='YlOrRd', 
            square=True, cbar_kws={'label': 'Similitud de Tanimoto'})
plt.title('Matriz de Similitud Molecular\n(Morgan Fingerprints, radio=2)', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Fármacos', fontsize=12)
plt.ylabel('Fármacos', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 4. Descriptores 3D (Dependen de conformación)

In [ ]:
def calcular_descriptores_3d(mol):
    """
    Calcula descriptores 3D que requieren coordenadas espaciales
    
    Nota: La molécula debe tener una conformación 3D embebida
    """
    # Verificar si tiene conformación 3D
    if mol.GetNumConformers() == 0:
        # Generar conformación 3D
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, randomSeed=42)
        AllChem.MMFFOptimizeMolecule(mol)
    
    descriptores = {
        # Momentos de inercia principales (PMI)
        'PMI1': Descriptors.PMI1(mol),
        'PMI2': Descriptors.PMI2(mol),
        'PMI3': Descriptors.PMI3(mol),
        
        # Ratios de PMI (forma molecular)
        'NPR1': Descriptors.NPR1(mol),  # Normalized PMI ratio 1
        'NPR2': Descriptors.NPR2(mol),  # Normalized PMI ratio 2
        
        # Radio de giro (compacidad)
        'RadiusOfGyration': Descriptors.RadiusOfGyration(mol),
        
        # Asphericity (desviación de forma esférica)
        'Asphericity': Descriptors.Asphericity(mol),
        
        # Eccentricity (elongación)
        'Eccentricity': Descriptors.Eccentricity(mol),
        
        # InertialShapeFactor
        'InertialShapeFactor': Descriptors.InertialShapeFactor(mol),
        
        # SpherocityIndex
        'SpherocityIndex': Descriptors.SpherocityIndex(mol),
    }
    
    return descriptores

# Calcular descriptores 3D para los fármacos
print("Descriptores 3D:\n")
desc_3d_data = []

for nombre, smiles in farmacos:
    mol = Chem.MolFromSmiles(smiles)
    desc_3d = calcular_descriptores_3d(mol)
    desc_3d['Nombre'] = nombre
    desc_3d_data.append(desc_3d)

df_3d = pd.DataFrame(desc_3d_data)
cols = ['Nombre'] + [col for col in df_3d.columns if col != 'Nombre']
df_3d = df_3d[cols]
df_3d

### 4.1 Interpretación de Descriptores 3D

**Momentos de Inercia (PMI)**:
- Describen la distribución de masa en el espacio 3D
- PMI1 ≤ PMI2 ≤ PMI3

**NPR1 y NPR2** (Normalized PMI Ratios):
- **Rod-like**: NPR1 ≈ 0, NPR2 ≈ 0.5 (lineal)
- **Disc-like**: NPR1 ≈ 0.5, NPR2 ≈ 0.5 (plana)
- **Sphere-like**: NPR1 ≈ 1, NPR2 ≈ 1 (esférica)

**Asphericity**: 
- 0 = perfectamente esférica
- Valores altos = elongada/aplanada

**Eccentricity**:
- Mide la elongación molecular

In [ ]:
# Visualizar forma molecular en diagrama PMI
plt.figure(figsize=(10, 8))

# Triángulo PMI
# Vértices: Rod (0, 0.5), Disc (0.5, 0.5), Sphere (1, 1)
triangle = np.array([[0, 0.5], [0.5, 0.5], [1, 1], [0, 0.5]])
plt.plot(triangle[:, 0], triangle[:, 1], 'k--', linewidth=2, alpha=0.3)

# Graficar moléculas
for _, row in df_3d.iterrows():
    plt.scatter(row['NPR1'], row['NPR2'], s=200, alpha=0.7, edgecolors='black', linewidth=2)
    plt.annotate(row['Nombre'], (row['NPR1'], row['NPR2']), 
                xytext=(5, 5), textcoords='offset points', fontsize=11, fontweight='bold')

# Etiquetas de formas
plt.text(0, 0.5, 'Rod\n(Lineal)', ha='right', va='center', fontsize=10, 
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
plt.text(0.5, 0.5, 'Disc\n(Plana)', ha='center', va='bottom', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
plt.text(1, 1, 'Sphere\n(Esférica)', ha='left', va='center', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

plt.xlabel('NPR1 (I₁/I₃)', fontsize=12, fontweight='bold')
plt.ylabel('NPR2 (I₂/I₃)', fontsize=12, fontweight='bold')
plt.title('Diagrama de Forma Molecular (PMI)\nDistribución Rod-Disc-Sphere', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlim(-0.1, 1.1)
plt.ylim(0.4, 1.1)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Cálculo Masivo de Descriptores

RDKit proporciona más de **200 descriptores** predefinidos. Podemos calcularlos todos simultáneamente.

In [ ]:
def calcular_todos_descriptores(smiles_list, nombres):
    """
    Calcula todos los descriptores disponibles en RDKit
    """
    # Obtener lista de todos los descriptores
    desc_names = [desc[0] for desc in Descriptors.descList]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    
    data = []
    for nombre, smiles in zip(nombres, smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            # Calcular todos los descriptores
            desc_vals = calc.CalcDescriptors(mol)
            row = {'Nombre': nombre, 'SMILES': smiles}
            row.update(dict(zip(desc_names, desc_vals)))
            data.append(row)
    
    return pd.DataFrame(data)

# Calcular todos los descriptores
nombres = [nombre for nombre, _ in farmacos]
smiles_list = [smiles for _, smiles in farmacos]

df_completo = calcular_todos_descriptores(smiles_list, nombres)

print(f"Total de descriptores calculados: {len(df_completo.columns) - 2}")
print(f"\nPrimeras 10 columnas:")
df_completo.iloc[:, :10]

In [ ]:
# Seleccionar descriptores más relevantes para análisis
descriptores_clave = ['Nombre', 'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 
                      'TPSA', 'NumRotatableBonds', 'NumAromaticRings', 
                      'FractionCsp3', 'BertzCT', 'Chi0v', 'Kappa1']

df_seleccionado = df_completo[descriptores_clave]
print("Descriptores Clave Seleccionados:\n")
df_seleccionado

## 6. Análisis de Componentes Principales (PCA)

In [ ]:
# Preparar datos para PCA (eliminar columnas no numéricas)
X = df_completo.drop(['Nombre', 'SMILES'], axis=1)

# Eliminar descriptores con valores NaN o infinitos
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.mean())

# Estandarizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Crear DataFrame con resultados
df_pca = pd.DataFrame({
    'Nombre': nombres,
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1]
})

print(f"Varianza explicada por PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"Varianza explicada por PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum():.2%}")

df_pca

In [ ]:
# Visualizar PCA
plt.figure(figsize=(10, 7))

plt.scatter(df_pca['PC1'], df_pca['PC2'], s=300, alpha=0.6, 
           c=range(len(df_pca)), cmap='viridis', edgecolors='black', linewidth=2)

for _, row in df_pca.iterrows():
    plt.annotate(row['Nombre'], (row['PC1'], row['PC2']), 
                xytext=(8, 8), textcoords='offset points', 
                fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7))

plt.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.axvline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)', 
          fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)', 
          fontsize=12, fontweight='bold')
plt.title('Análisis de Componentes Principales (PCA)\nEspacio de Descriptores Moleculares', 
         fontsize=14, fontweight='bold', pad=15)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Propiedades Fisicoquímicas Estimadas

### 7.1 Coeficiente de Partición (LogP)

El **LogP** mide la lipofilia (afinidad por lípidos vs agua):
- LogP < 0: Hidrófila
- LogP > 0: Lipófila
- Óptimo para fármacos: 1-3

In [ ]:
def calcular_propiedades_fisicoquimicas(mol):
    """
    Calcula propiedades fisicoquímicas relevantes para fármacos
    """
    return {
        'LogP (Wildman-Crippen)': Descriptors.MolLogP(mol),
        'MR (Refracción Molar)': Descriptors.MolMR(mol),
        'TPSA (Å²)': Descriptors.TPSA(mol),
        'LabuteASA': Descriptors.LabuteASA(mol),
        'Carga Parcial (+)': sum(abs(a.GetDoubleProp('_GasteigerCharge')) 
                                 for a in mol.GetAtoms() 
                                 if a.HasProp('_GasteigerCharge') 
                                 and a.GetDoubleProp('_GasteigerCharge') > 0),
    }

# Calcular para los fármacos
print("Propiedades Fisicoquímicas:\n")
prop_data = []

for nombre, smiles in farmacos:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.ComputeGasteigerCharges(mol)  # Calcular cargas parciales
    props = calcular_propiedades_fisicoquimicas(mol)
    props['Nombre'] = nombre
    prop_data.append(props)

df_props = pd.DataFrame(prop_data)
cols = ['Nombre'] + [col for col in df_props.columns if col != 'Nombre']
df_props = df_props[cols]
df_props

### 7.2 Estimación de Solubilidad y QED

**QED** (Quantitative Estimate of Drug-likeness): Puntuación 0-1 que mide qué tan similar es una molécula a un fármaco.

In [ ]:
def calcular_qed_druglikeness(mol):
    """
    Calcula el QED (Quantitative Estimate of Drug-likeness)
    """
    qed_score = QED.qed(mol)
    
    # Interpretación
    if qed_score >= 0.7:
        categoria = "Excelente"
    elif qed_score >= 0.5:
        categoria = "Bueno"
    elif qed_score >= 0.3:
        categoria = "Moderado"
    else:
        categoria = "Pobre"
    
    return qed_score, categoria

# Calcular QED
print("Drug-likeness Score (QED):\n")
print(f"{'Fármaco':<15} {'QED Score':<12} {'Categoría':<15}")
print("="*45)

qed_data = []
for nombre, smiles in farmacos:
    mol = Chem.MolFromSmiles(smiles)
    qed_score, categoria = calcular_qed_druglikeness(mol)
    print(f"{nombre:<15} {qed_score:<12.3f} {categoria:<15}")
    qed_data.append({'Nombre': nombre, 'QED': qed_score, 'Categoría': categoria})

df_qed = pd.DataFrame(qed_data)

In [ ]:
# Visualizar QED scores
plt.figure(figsize=(10, 6))

colors = ['green' if q >= 0.7 else 'orange' if q >= 0.5 else 'red' 
         for q in df_qed['QED']]

bars = plt.bar(df_qed['Nombre'], df_qed['QED'], color=colors, 
              alpha=0.7, edgecolor='black', linewidth=2)

# Líneas de referencia
plt.axhline(0.7, color='green', linestyle='--', linewidth=2, 
           alpha=0.5, label='Excelente (≥0.7)')
plt.axhline(0.5, color='orange', linestyle='--', linewidth=2, 
           alpha=0.5, label='Bueno (≥0.5)')
plt.axhline(0.3, color='red', linestyle='--', linewidth=2, 
           alpha=0.5, label='Moderado (≥0.3)')

# Añadir valores sobre las barras
for bar, qed in zip(bars, df_qed['QED']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{qed:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.ylabel('QED Score', fontsize=12, fontweight='bold')
plt.xlabel('Fármaco', fontsize=12, fontweight='bold')
plt.title('Quantitative Estimate of Drug-likeness (QED)\nPuntuación de Similitud a Fármacos', 
         fontsize=14, fontweight='bold', pad=15)
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=10)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 8. Aplicación: Mini-Análisis QSAR

In [ ]:
# Conjunto de datos ejemplo: Actividad antiinflamatoria (IC50, μM)
# Valores ficticios para demostración
actividad_biologica = {
    'Aspirina': 150.0,
    'Ibuprofeno': 8.5,
    'Paracetamol': 220.0,
    'Cafeína': 450.0,
}

# Convertir a pIC50 (log-transform para linearizar)
# pIC50 = -log10(IC50 en M)
df_qsar = df_seleccionado.copy()
df_qsar['IC50 (μM)'] = df_qsar['Nombre'].map(actividad_biologica)
df_qsar['pIC50'] = -np.log10(df_qsar['IC50 (μM)'] * 1e-6)

print("Dataset QSAR:\n")
df_qsar[['Nombre', 'IC50 (μM)', 'pIC50', 'MolWt', 'MolLogP', 'TPSA']]

In [ ]:
# Correlaciones entre descriptores y actividad
descriptores_num = ['MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 
                    'TPSA', 'NumRotatableBonds']

correlaciones = []
for desc in descriptores_num:
    corr = df_qsar[[desc, 'pIC50']].corr().iloc[0, 1]
    correlaciones.append({'Descriptor': desc, 'Correlación': corr})

df_corr = pd.DataFrame(correlaciones).sort_values('Correlación', 
                                                  key=abs, ascending=False)

print("\nCorrelaciones con Actividad Biológica (pIC50):\n")
print(df_corr.to_string(index=False))

In [ ]:
# Gráficos de correlación
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, desc in enumerate(descriptores_num):
    ax = axes[i]
    
    # Scatter plot
    ax.scatter(df_qsar[desc], df_qsar['pIC50'], s=150, alpha=0.6, 
              edgecolors='black', linewidth=2)
    
    # Añadir nombres
    for _, row in df_qsar.iterrows():
        ax.annotate(row['Nombre'][:3], (row[desc], row['pIC50']), 
                   xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # Línea de tendencia
    z = np.polyfit(df_qsar[desc], df_qsar['pIC50'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df_qsar[desc].min(), df_qsar[desc].max(), 100)
    ax.plot(x_line, p(x_line), 'r--', alpha=0.7, linewidth=2)
    
    # Correlación
    corr = df_qsar[[desc, 'pIC50']].corr().iloc[0, 1]
    ax.text(0.05, 0.95, f'r = {corr:.3f}', transform=ax.transAxes, 
           fontsize=11, fontweight='bold', va='top',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel(desc, fontsize=11, fontweight='bold')
    ax.set_ylabel('pIC50 (Actividad)', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Correlaciones Descriptor-Actividad (Análisis QSAR)', 
            fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 9. Ejercicios Prácticos

### Ejercicio 1: Análisis de una Serie de Compuestos

Analiza los siguientes fármacos antihistamínicos:

**Tarea:**
1. Calcula descriptores básicos (MW, LogP, HBD, HBA, TPSA)
2. Evalúa la Regla de Lipinski
3. Calcula QED scores
4. Identifica el compuesto más "drug-like"
5. Genera una matriz de similitud usando Morgan fingerprints

In [ ]:
# TU CÓDIGO AQUÍ

antihistaminicos = [
    ('Cetirizina', 'O=C(O)COCCN1CCN(CC1)C(c2ccc(Cl)cc2)c3ccccc3'),
    ('Loratadina', 'CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc4cccnc24)CC1'),
    ('Difenhidramina', 'CN(C)CCOC(c1ccccc1)c2ccccc2'),
    ('Clorfeniramina', 'CN(C)CCC(c1ccc(Cl)cc1)c2ccccn2'),
]

# Tu análisis aquí

### Ejercicio 2: Exploración de Descriptores 3D

Compara las formas moleculares usando descriptores 3D.

**Tarea:**
1. Genera conformaciones 3D para: benceno, ciclohexano, adamantano
2. Calcula descriptores de forma (PMI, NPR1, NPR2, Asphericity)
3. Ubica las moléculas en el diagrama PMI (Rod-Disc-Sphere)
4. Interpreta qué moléculas son más esféricas, planas o lineales
5. Calcula el radio de giro de cada una

In [ ]:
# TU CÓDIGO AQUÍ

moleculas_forma = [
    ('Benceno', 'c1ccccc1'),
    ('Ciclohexano', 'C1CCCCC1'),
    ('Adamantano', 'C1C2CC3CC1CC(C2)C3'),
]

# Tu análisis aquí

### Ejercicio 3: Comparación de Fingerprints

Compara diferentes tipos de fingerprints.

**Tarea:**
1. Toma 2 moléculas similares y 1 diferente
2. Calcula similitud con: Morgan, MACCS, RDKit fingerprints
3. Compara los resultados entre los 3 métodos
4. ¿Qué método detecta mejor la similitud estructural?
5. Visualiza las matrices de similitud como heatmaps

In [ ]:
# TU CÓDIGO AQUÍ

moleculas_test = [
    ('Tolueno', 'Cc1ccccc1'),
    ('Xileno', 'Cc1ccccc1C'),
    ('Piridina', 'c1ccncc1'),
]

# Tu análisis aquí

### Ejercicio 4: Predicción de Propiedades

Usa descriptores para predecir permeabilidad.

**Tarea:**
1. Dataset: 5 moléculas con permeabilidad conocida (valores ficticios)
2. Calcula 10 descriptores relevantes
3. Identifica correlaciones con permeabilidad
4. Crea un modelo lineal simple (regresión)
5. Predice la permeabilidad de una molécula nueva

In [ ]:
# TU CÓDIGO AQUÍ

# Dataset ejemplo
dataset_permeabilidad = {
    'CCCO': 8.5,           # 1-propanol
    'CC(=O)O': 6.2,        # Ácido acético  
    'c1ccccc1O': 7.8,      # Fenol
    'CCN(CC)CC': 9.1,      # Trietilamina
    'CC(C)O': 8.0,         # Isopropanol
}

# Molécula nueva para predecir
nueva_molecula = 'CCCCO'  # 1-butanol

# Tu análisis aquí

### Ejercicio 5: Análisis Multivariado

Realiza un análisis PCA completo.

**Tarea:**
1. Usa un conjunto de 10 moléculas diversas
2. Calcula todos los descriptores 2D disponibles
3. Aplica PCA y reduce a 3 componentes
4. Visualiza en gráfico 3D interactivo
5. Identifica qué descriptores contribuyen más a PC1 y PC2
6. Agrupa moléculas por similitud usando los componentes principales

In [ ]:
# TU CÓDIGO AQUÍ

moleculas_diversas = [
    ('Metano', 'C'),
    ('Etanol', 'CCO'),
    ('Benceno', 'c1ccccc1'),
    ('Aspirina', 'CC(=O)Oc1ccccc1C(=O)O'),
    ('Glucosa', 'C(C1C(C(C(C(O1)O)O)O)O)O'),
    ('Cafeína', 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'),
    ('Colesterol', 'CC(C)CCCC(C)C1CCC2C1(CCC3C2CC=C4C3(CCC(C4)O)C)C'),
    ('Alanina', 'CC(C(=O)O)N'),
    ('Urea', 'C(=O)(N)N'),
    ('Piridina', 'c1ccncc1'),
]

# Tu análisis aquí

## Resumen y Conclusiones

En esta actividad has aprendido a:

✅ **Calcular descriptores moleculares** 2D y 3D con RDKit  
✅ **Evaluar drug-likeness** usando Regla de Lipinski y QED  
✅ **Generar fingerprints** moleculares para análisis de similitud  
✅ **Analizar propiedades fisicoquímicas** (LogP, TPSA, solubilidad)  
✅ **Aplicar análisis multivariado** (PCA) para reducción dimensional  
✅ **Realizar análisis QSAR** correlacionando estructura con actividad  
✅ **Visualizar e interpretar** resultados de forma efectiva  

### Puntos Clave

- Los descriptores codifican información estructural en valores numéricos
- Descriptores 2D son rápidos y no requieren conformación 3D
- Descriptores 3D capturan información geométrica importante
- Fingerprints permiten comparaciones rápidas de similitud
- La Regla de Lipinski predice biodisponibilidad oral
- QED es una métrica integrada de drug-likeness
- PCA ayuda a visualizar relaciones en espacios de alta dimensionalidad
- QSAR correlaciona descriptores con actividad biológica

### Aplicaciones en Drug Discovery

1. **Virtual Screening**: Filtrar grandes bibliotecas por propiedades deseadas
2. **Lead Optimization**: Mejorar propiedades fisicoquímicas
3. **ADMET Prediction**: Predecir absorción, distribución, metabolismo, excreción, toxicidad
4. **Similarity Search**: Encontrar análogos estructurales
5. **QSAR Modeling**: Predecir actividad de nuevos compuestos

### Próximos Pasos

- **Actividad 2.7:** Búsqueda y descarga de estructuras desde bases de datos
- **Módulo 3:** Mecánica Molecular y campos de fuerza

---

## Referencias

1. Lipinski, C. A. et al. (1997). *Experimental and computational approaches to estimate solubility and permeability.* Adv. Drug Deliv. Rev., 23, 3-25.

2. Bickerton, G. R. et al. (2012). *Quantifying the chemical beauty of drugs.* Nat. Chem., 4, 90-98.

3. Rogers, D., & Hahn, M. (2010). *Extended-connectivity fingerprints.* J. Chem. Inf. Model., 50(5), 742-754.

4. Todeschini, R., & Consonni, V. (2009). *Molecular Descriptors for Chemoinformatics.* Wiley-VCH.

5. RDKit Documentation: [Descriptor Calculation](https://www.rdkit.org/docs/GettingStartedInPython.html#molecular-descriptors)

6. Sauer, W. H., & Schwarz, M. K. (2003). *Molecular shape diversity of combinatorial libraries.* J. Chem. Inf. Comput. Sci., 43(3), 987-1003.

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 2.6: Descriptores Moleculares y Propiedades Calculadas**

**Siguiente actividad**: Búsqueda y Descarga de Estructuras (PubChem, PDB)

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_2.5-Conformaciones-blue.svg)](05_conformaciones.ipynb)
[![Siguiente](https://img.shields.io/badge/Actividad_2.7_➡️-Bases_de_Datos-green.svg)](07_bases_datos.ipynb)

---

📚 **[Volver al Módulo 2](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>